In [1]:
'''
Preprocess script for SNPs, mostly covering the translation part
synonomous vs non-synonomous
the v2 refers to an alternative scoring method. In this case, we're just going to ignore the frameshifts. 
'''
import pandas as pd, numpy as np, re, sys, json
from pathlib import Path
from multiprocessing import Pool
from collections import Counter

in_path = Path('/d/data/plasmo/nat_out/v2')
in_file = in_path / 'snps_by_genes.tsv'
out_path = in_path / 'gene_scores.tsv'
codons_file = in_path / 'codons.json'
genes_file = in_path / 'filtered_genes.txt'
ref_file = in_path / '3d7.fasta'
sift_file = in_path / 'sift.tsv'

In [2]:
#loading the actual SNPs
in_df = pd.read_csv(in_file, sep='\t', index_col = [0,1])
sift_df = pd.read_csv(sift_file, sep='\t', index_col = [0,1])

In [3]:
by_genes = in_df.groupby('CHROM') #each iteration yields (name, df)

In [4]:
#function for assigning score to each element of each row
def toScore(row):
    #the stipulation here is that the 'ref' are the 5 reference bases surrounding the SNP (+2, -2)
    #scoring scheme: regular = 1, stop codon or indel = 3.
    def getSingleScore(snp, ref, alts, sift_info):       
        
#         print('calling on ', snp, ref, alts)
        #ref may be a list
        if snp in ref:
            return 0 #no mutation
        
        if snp not in alts:
            return 0 #no annotation
        
#         print('scoring ', snp, ref, alts, sift_info)
        #actual scoring
        row = sift_info.loc[snp]
        
        if len(row.shape) > 1:
            row = row.head(1)
        
        if re.search('DELETERIOUS', str(row['SIFT_PREDICTION'])):
            return 1
        
#         if re.search('TOLERATED', str(row['SIFT_PREDICTION'])):
#             return 0
        
#         if re.search('^FRAMESHIFT', str(row['VARIANT_TYPE'])):
#             return 2
        
#         if re.search('STOP', str(row['VARIANT_TYPE'])):
#             return 2
        
#         if re.search('SUBSTITUTION', str(row['VARIANT_TYPE'])):
#             return 2
        
#         if re.search('NONSYNONYMOUS', str(row['VARIANT_TYPE'])):
#             return 1
        
        return 0
    
    def getSingleScoreWrapper(snp, ref, alts, sift_info):

        score = getSingleScore(snp, ref, alts, sift_info)

        return score
    
    def zeroScore(x):
        return 0
    
    global sift_df
       
    #the name of the chromosome should be id:chromosome
    full_id = row.name[0].split(':')
    id = full_id[0]
    chr = full_id[1]
    pos = int(row.name[1])
    
    try:
        #have annotation
        print('annotation at', chr, pos)
        sift_info = sift_df.loc[(chr, pos)]
    except:
        #no annotation
        print('no annotation at ', chr, pos)
        return row.apply(zeroScore)
    
    
    ref = sift_info['REF_ALLELE'].values
    alts = sift_info['ALT_ALLELE'].values
    sift_info.set_index('ALT_ALLELE', inplace = True)
    
    scores = row.apply(getSingleScoreWrapper, 1, args = (ref, alts, sift_info))

    return scores

def worker(args):
    #hard coded args
    name = args[0]
    snps_df = args[1]
    
    scores = snps_df.apply(toScore, 1)
    result = scores.apply(sum, 0)
    result = pd.DataFrame(result, columns = [name]).T
    return result

with Pool(6) as pool:
    res_dfs = pool.map(worker, by_genes)
res = pd.concat(res_dfs)

annotation at Pf3D7_01_v3 98866
annotation at Pf3D7_01_v3 98868
annotation at Pf3D7_01_v3 98917
annotation at Pf3D7_01_v3 98971
annotation at Pf3D7_01_v3 98978
annotation at Pf3D7_01_v3 99018
no annotation at  Pf3D7_01_v3 99018
annotation at Pf3D7_01_v3 99107
no annotation at  Pf3D7_01_v3 99107
annotation at Pf3D7_01_v3 99144
no annotation at  Pf3D7_01_v3 99144
annotation at Pf3D7_01_v3 99147
no annotation at  Pf3D7_01_v3 99147
annotation at Pf3D7_01_v3 99187
no annotation at  Pf3D7_01_v3 99187
annotation at Pf3D7_01_v3 99226
annotation at Pf3D7_01_v3 99273
annotation at Pf3D7_01_v3 99395
annotation at Pf3D7_01_v3 99723
annotation at Pf3D7_01_v3 99861
annotation at Pf3D7_01_v3 99870
annotation at Pf3D7_01_v3 100315
annotation at Pf3D7_01_v3 100325
annotation at Pf3D7_01_v3 100330
annotation at Pf3D7_01_v3 100333
annotation at Pf3D7_01_v3 100345
annotation at Pf3D7_01_v3 100362
annotation at Pf3D7_01_v3 100418
annotation at Pf3D7_01_v3 100431
annotation at Pf3D7_01_v3 100461
annotation 

annotation at Pf3D7_01_v3 138966
annotation at Pf3D7_01_v3 139023
annotation at Pf3D7_01_v3 139056
annotation at Pf3D7_01_v3 139071
annotation at Pf3D7_01_v3 139134
annotation at Pf3D7_01_v3 139187
annotation at Pf3D7_01_v3 139191
annotation at Pf3D7_01_v3 140820
annotation at Pf3D7_01_v3 141126
no annotation at  Pf3D7_01_v3 141126
annotation at Pf3D7_01_v3 141128
no annotation at  Pf3D7_01_v3 141128
annotation at Pf3D7_01_v3 141134
no annotation at  Pf3D7_01_v3 141134
annotation at Pf3D7_01_v3 141140
no annotation at  Pf3D7_01_v3 141140
annotation at Pf3D7_01_v3 141144
no annotation at  Pf3D7_01_v3 141144
annotation at Pf3D7_01_v3 141146
no annotation at  Pf3D7_01_v3 141146
annotation at Pf3D7_01_v3 141147
no annotation at  Pf3D7_01_v3 141147
annotation at Pf3D7_01_v3 141150
no annotation at  Pf3D7_01_v3 141150
annotation at Pf3D7_01_v3 141242
no annotation at  Pf3D7_01_v3 141242
annotation at Pf3D7_01_v3 141244
no annotation at  Pf3D7_01_v3 141244
annotation at Pf3D7_01_v3 141257
no 

annotation at Pf3D7_01_v3 180554
annotation at Pf3D7_01_v3 180680
annotation at Pf3D7_01_v3 182845
annotation at Pf3D7_01_v3 182876
annotation at Pf3D7_01_v3 182927
annotation at Pf3D7_01_v3 183260
annotation at Pf3D7_01_v3 183473
annotation at Pf3D7_01_v3 183682
annotation at Pf3D7_01_v3 183805
annotation at Pf3D7_01_v3 183828
annotation at Pf3D7_01_v3 183833
annotation at Pf3D7_01_v3 183839
annotation at Pf3D7_01_v3 183843
annotation at Pf3D7_01_v3 183844
annotation at Pf3D7_01_v3 183848
annotation at Pf3D7_01_v3 183864
annotation at Pf3D7_01_v3 183986
annotation at Pf3D7_01_v3 184016
annotation at Pf3D7_01_v3 184114
annotation at Pf3D7_01_v3 190271
annotation at Pf3D7_01_v3 190317
annotation at Pf3D7_01_v3 190418
annotation at Pf3D7_01_v3 190595
annotation at Pf3D7_01_v3 190616
annotation at Pf3D7_01_v3 190636
annotation at Pf3D7_01_v3 190739
annotation at Pf3D7_01_v3 190777
annotation at Pf3D7_01_v3 190821
annotation at Pf3D7_01_v3 190862
annotation at Pf3D7_01_v3 190944
annotation

annotation at Pf3D7_01_v3 225393
annotation at Pf3D7_01_v3 225473
annotation at Pf3D7_01_v3 230331
annotation at Pf3D7_01_v3 230468
annotation at Pf3D7_01_v3 230489
annotation at Pf3D7_01_v3 230504
annotation at Pf3D7_01_v3 230519
annotation at Pf3D7_01_v3 230581
annotation at Pf3D7_01_v3 230590
annotation at Pf3D7_01_v3 230718
annotation at Pf3D7_01_v3 230754
annotation at Pf3D7_01_v3 230828
annotation at Pf3D7_01_v3 230930
annotation at Pf3D7_01_v3 231024
annotation at Pf3D7_01_v3 231063
annotation at Pf3D7_01_v3 231150
annotation at Pf3D7_01_v3 231384
no annotation at  Pf3D7_01_v3 231384
annotation at Pf3D7_01_v3 231442
no annotation at  Pf3D7_01_v3 231442
annotation at Pf3D7_01_v3 231461
no annotation at  Pf3D7_01_v3 231461
annotation at Pf3D7_01_v3 231465
no annotation at  Pf3D7_01_v3 231465
annotation at Pf3D7_01_v3 231469
no annotation at  Pf3D7_01_v3 231469
annotation at Pf3D7_01_v3 231531
no annotation at  Pf3D7_01_v3 231531
annotation at Pf3D7_01_v3 231587
no annotation at  P

annotation at Pf3D7_01_v3 283173
annotation at Pf3D7_01_v3 283174
annotation at Pf3D7_01_v3 283178
annotation at Pf3D7_01_v3 283194
annotation at Pf3D7_01_v3 283195
annotation at Pf3D7_01_v3 283198
annotation at Pf3D7_01_v3 283244
annotation at Pf3D7_01_v3 283247
annotation at Pf3D7_01_v3 283249
annotation at Pf3D7_01_v3 283251
annotation at Pf3D7_01_v3 283252
annotation at Pf3D7_01_v3 283260
annotation at Pf3D7_01_v3 283267
annotation at Pf3D7_01_v3 283276
annotation at Pf3D7_01_v3 283279
annotation at Pf3D7_01_v3 283282
annotation at Pf3D7_01_v3 283296
annotation at Pf3D7_01_v3 283299
annotation at Pf3D7_01_v3 283302
annotation at Pf3D7_01_v3 283309
annotation at Pf3D7_01_v3 283320
annotation at Pf3D7_01_v3 283335
annotation at Pf3D7_01_v3 283344
annotation at Pf3D7_01_v3 283345
annotation at Pf3D7_01_v3 283351
annotation at Pf3D7_01_v3 283390
annotation at Pf3D7_01_v3 283391
annotation at Pf3D7_01_v3 283399
annotation at Pf3D7_01_v3 283405
annotation at Pf3D7_01_v3 283406
annotation

...

annotation at Pf3D7_14_v3 2883918
no annotation at  Pf3D7_14_v3 2883918
annotation at Pf3D7_14_v3 2883921
no annotation at  Pf3D7_14_v3 2883921
annotation at Pf3D7_14_v3 2883931
no annotation at  Pf3D7_14_v3 2883931
annotation at Pf3D7_14_v3 2884046
annotation at Pf3D7_14_v3 2884241
annotation at Pf3D7_14_v3 2884256
annotation at Pf3D7_14_v3 2884505
annotation at Pf3D7_14_v3 2884512
annotation at Pf3D7_14_v3 2884572
annotation at Pf3D7_14_v3 2885144
annotation at Pf3D7_14_v3 2885310
annotation at Pf3D7_14_v3 2885415
annotation at Pf3D7_14_v3 2885532
no annotation at  Pf3D7_14_v3 2885532
annotation at Pf3D7_14_v3 2885908
annotation at Pf3D7_14_v3 2898116
annotation at Pf3D7_14_v3 2898244
no annotation at  Pf3D7_14_v3 2898244
annotation at Pf3D7_14_v3 2898284
no annotation at  Pf3D7_14_v3 2898284
annotation at Pf3D7_14_v3 2898308
no annotation at  Pf3D7_14_v3 2898308
annotation at Pf3D7_14_v3 2898520
annotation at Pf3D7_14_v3 2898555
annotation at Pf3D7_14_v3 2902451
annotation at Pf3D7_

annotation at Pf3D7_14_v3 2948916
annotation at Pf3D7_14_v3 2948924
annotation at Pf3D7_14_v3 2948926
annotation at Pf3D7_14_v3 2948927
annotation at Pf3D7_14_v3 2948930
annotation at Pf3D7_14_v3 2948931
annotation at Pf3D7_14_v3 2948934
annotation at Pf3D7_14_v3 2948940
annotation at Pf3D7_14_v3 2948942
annotation at Pf3D7_14_v3 2948944
annotation at Pf3D7_14_v3 2948948
annotation at Pf3D7_14_v3 2948967
annotation at Pf3D7_14_v3 2948985
annotation at Pf3D7_14_v3 2949038
annotation at Pf3D7_14_v3 2949039
annotation at Pf3D7_14_v3 2949052
annotation at Pf3D7_14_v3 2949056
annotation at Pf3D7_14_v3 2949193
annotation at Pf3D7_14_v3 2949248
annotation at Pf3D7_14_v3 2949298
annotation at Pf3D7_14_v3 2949672
annotation at Pf3D7_14_v3 2949918
annotation at Pf3D7_14_v3 2950291
annotation at Pf3D7_14_v3 2950330
annotation at Pf3D7_14_v3 2950344
annotation at Pf3D7_14_v3 2950481
annotation at Pf3D7_14_v3 2950552
annotation at Pf3D7_14_v3 2950584
annotation at Pf3D7_14_v3 2950600
annotation at 

annotation at Pf3D7_14_v3 3063911
annotation at Pf3D7_14_v3 3063912
annotation at Pf3D7_14_v3 3063915
annotation at Pf3D7_14_v3 3063924
annotation at Pf3D7_14_v3 3063992
annotation at Pf3D7_14_v3 3064181
annotation at Pf3D7_14_v3 3064219
annotation at Pf3D7_14_v3 3064253
annotation at Pf3D7_14_v3 3064270
annotation at Pf3D7_14_v3 3064279
annotation at Pf3D7_14_v3 3064393
annotation at Pf3D7_14_v3 3064397
annotation at Pf3D7_14_v3 3064484
annotation at Pf3D7_14_v3 3064495
annotation at Pf3D7_14_v3 3064535
annotation at Pf3D7_14_v3 3064633
annotation at Pf3D7_14_v3 3064686
annotation at Pf3D7_14_v3 3064757
annotation at Pf3D7_14_v3 3064822
annotation at Pf3D7_14_v3 3065095
annotation at Pf3D7_14_v3 3065105
annotation at Pf3D7_14_v3 3065173
annotation at Pf3D7_14_v3 3065215
annotation at Pf3D7_14_v3 3065335
annotation at Pf3D7_14_v3 3067796
annotation at Pf3D7_14_v3 3067818
no annotation at  Pf3D7_14_v3 3067818
annotation at Pf3D7_14_v3 3067911
no annotation at  Pf3D7_14_v3 3067911
annota

annotation at Pf3D7_14_v3 3103114
annotation at Pf3D7_14_v3 3103179
annotation at Pf3D7_14_v3 3103299
annotation at Pf3D7_14_v3 3103324
annotation at Pf3D7_14_v3 3103362
annotation at Pf3D7_14_v3 3103795
annotation at Pf3D7_14_v3 3103816
annotation at Pf3D7_14_v3 3103926
annotation at Pf3D7_14_v3 3104038
annotation at Pf3D7_14_v3 3104136
annotation at Pf3D7_14_v3 3104140
annotation at Pf3D7_14_v3 3104314
annotation at Pf3D7_14_v3 3104647
no annotation at  Pf3D7_14_v3 3104647
annotation at Pf3D7_14_v3 3104649
no annotation at  Pf3D7_14_v3 3104649
annotation at Pf3D7_14_v3 3104653
no annotation at  Pf3D7_14_v3 3104653
annotation at Pf3D7_14_v3 3104657
no annotation at  Pf3D7_14_v3 3104657
annotation at Pf3D7_14_v3 3104663
no annotation at  Pf3D7_14_v3 3104663
annotation at Pf3D7_14_v3 3104673
no annotation at  Pf3D7_14_v3 3104673
annotation at Pf3D7_14_v3 3104700
no annotation at  Pf3D7_14_v3 3104700
annotation at Pf3D7_14_v3 3104755
annotation at Pf3D7_14_v3 3104847
annotation at Pf3D7_

no annotation at  Pf3D7_API_v3 17950
annotation at Pf3D7_API_v3 17955
no annotation at  Pf3D7_API_v3 17955
annotation at Pf3D7_API_v3 17984
no annotation at  Pf3D7_API_v3 17984
annotation at Pf3D7_API_v3 18225
no annotation at  Pf3D7_API_v3 18225
annotation at Pf3D7_API_v3 18510
no annotation at  Pf3D7_API_v3 18510
annotation at Pf3D7_API_v3 18591
no annotation at  Pf3D7_API_v3 18591
annotation at Pf3D7_API_v3 18603
no annotation at  Pf3D7_API_v3 18603
annotation at Pf3D7_API_v3 18633
no annotation at  Pf3D7_API_v3 18633
annotation at Pf3D7_API_v3 18636
no annotation at  Pf3D7_API_v3 18636
annotation at Pf3D7_API_v3 18761
no annotation at  Pf3D7_API_v3 18761
annotation at Pf3D7_API_v3 18772
no annotation at  Pf3D7_API_v3 18772
annotation at Pf3D7_API_v3 18927
no annotation at  Pf3D7_API_v3 18927
annotation at Pf3D7_API_v3 18939
no annotation at  Pf3D7_API_v3 18939
annotation at Pf3D7_API_v3 19259
no annotation at  Pf3D7_API_v3 19259
annotation at Pf3D7_API_v3 19412
no annotation at  Pf

In [5]:
res.T.to_csv(out_path, sep='\t')